# Tutorial 7

Extraing features out of networks.

In [ ]:
!pip3 install d2l

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms as T
from d2l import torch as d2l

## Loading Models and Datasets

Load a model and dataset from pytorch.

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
alexnet = models.alexnet(pretrained=True).to(device)
resnet152 = models.resnet152(pretrained=True).to(device)

In [ ]:
train_data = datasets.CIFAR10('train_data', download=True)
test_data = datasets.CIFAR10('test_data', download=True, train=False)

In [ ]:
train_data

In [ ]:
test_data

## Preprocess the data

The built in models expect the data to be formatted a very specific way.
* Need to be 3 channel images (assumed RGB in that order)
* Need to be at least 224 $\times$ 224 images.
* All scalar values need to be scaled to $[0, 1]$.
* Must then be $z$-scaled using means $[0.485, 0.456, 0.406]$ and standard deviations of $[0.229, 0.224, 0.225]$.

In [ ]:
batch_size = 256

# z-scale the data
normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])

transforms = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(), # also scales to [0, 1]
    normalize
])

train_data = datasets.CIFAR10('train_data', download=True, transform=transforms)
test_data = datasets.CIFAR10('test_data', download=True, train=False, transform=transforms)

data_loader_train = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

data_loader_test = DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=True
)

## Network Architecture

We can inspect the network's architecture with ```net._modules```

In [ ]:
alexnet._modules

In [ ]:
resnet152._modules

We can also use the `summary()` function.

In [ ]:
from torchsummary import summary

In [ ]:
summary(alexnet, input_size=(3, 224, 224))

In [ ]:
summary(resnet152, input_size=(3, 224, 224))

## Changing the Output Dimensions

You may notice the networks loaded have 1000 output elements. This is because they were trained on datasets that have 1000 classes. However, we are dealing with datasets of only 10 or 100 classes.

Hence, we need to change the output dimension shape.

In [ ]:
def init_weights(model):
  nn.init.normal_(model.weight, std=0.01)

out_layer = nn.Linear(4096, 10)
out_layer.apply(init_weights)

In [ ]:
alexnet2 = models.alexnet(pretrained=True).to(device)
alexnet2.classifier[6] = out_layer

In [ ]:
alexnet2._modules

Alternatively, you can also add an additional Linear layer.

In [ ]:
def init_weights(model):
  nn.init.normal_(model.weight, std=0.01)

out_layer = nn.Linear(1000, 10)
out_layer.apply(init_weights)

In [ ]:
alexnet2 = nn.Sequential(
    alexnet,
    out_layer
)

## Extracting Features

Sometimes, we only care about the features extracted mid-way through a network.

To extract features mid-way through a network, we need a hook and the name of the block.

### Extracting By Accessing Internal Blocks

In [ ]:
import torch.nn.functional as F
  
class SoftmaxRegression(nn.Module):
  def __init__(self, input_dim, output_dim, *args, **kwargs):
    super(SoftmaxRegression, self).__init__()
    self.layer = nn.Linear(input_dim, output_dim)

  def forward(self, X, *args, **kwargs):
    return F.softmax(self.layer(X), dim=-1)

softmax = SoftmaxRegression(9216, 10)

class NewModel(nn.Module):
  def __init__(self, pretrained_model, output_model, extract_layers):
    super(NewModel, self).__init__()
    self.pretrained = pretrained_model
    self.flatten = nn.Flatten()
    self.output = output_model
    self.inner_model = [getattr(self.pretrained, name) for name in extract_layers]
  
  def forward(self, x):
    for block in self.inner_model:
      x = block.forward(x)
    
    features = self.flatten.forward(x)
    return self.output.forward(features)

net = NewModel(alexnet, softmax, ['features'])

### Extracting Using Hooks

In [ ]:
import torch.nn.functional as F
  
class SoftmaxRegression(nn.Module):
  def __init__(self, input_dim, output_dim, *args, **kwargs):
    super(SoftmaxRegression, self).__init__()
    self.layer = nn.Linear(input_dim, output_dim)

  def forward(self, X, *args, **kwargs):
    return F.softmax(self.layer(X), dim=-1)

softmax = SoftmaxRegression(9216, 10)

class NewModel(nn.Module):
  def __init__(self, pretrained_model, output_model, extract_layer):
    super(NewModel, self).__init__()
    self.pretrained = pretrained_model
    self.flatten = nn.Flatten()
    self.output = output_model
    self.selected_out = {}
    self.extract_layer = extract_layer
    layer = getattr(self.pretrained, extract_layer)
    layer.register_forward_hook(self.get_features(extract_layer))

  def get_features(self, layer_name):
    def hook(block, input, output):
      self.selected_out[layer_name] = output
    return hook
  
  def forward(self, x):
    out = self.pretrained.forward(x)
    features = self.selected_out[self.extract_layer]
    features2 = self.flatten.forward(features)
    out2 = self.output.forward(features2)
    return out2

net = NewModel(alexnet, softmax, 'features')

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01, weight_decay=0.001)

In [ ]:
epochs = 5
d2l.train_ch3(net, data_loader_train, data_loader_test, loss, epochs, optimizer)